## Before coding

Create a new project

https://console.developers.google.com/projectcreate

Once you have created the project, enable access to the YouTube Data API

https://console.developers.google.com/apis/library

Once enabled, it is important that you get credentials for your project

https://console.developers.google.com/apis/credentials/wizard?api=youtube.googleapis.com

From the options select:  

| Option  | Value |
| ------------- | ------------- |
| ¿Qué API estás usando?  | **YouTube Data API v3**  |
| ¿Desde dónde llamarás a la API? | **Servidor Web**  |
| ¿A qué tipo de datos accederás? | **Datos públicos**  |  

Having selected such values, press: **"¿Qué credenciales necesito?"**  and you will be given an alphanumeric string that is your API key, place this value into the `api_key` variable:

In [ ]:
api_key = "" # Place your YT api key here
assert api_key != ""

## Now, coding

Import the necessary packages

In [ ]:
import requests
import json
import urllib
import isodate
import os
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from slugify import slugify
from pytube import YouTube

In [ ]:
search_url = "https://www.googleapis.com/youtube/v3/search?"
caption_url = "https://www.youtube.com/api/timedtext?"

In [ ]:
language_preferences = ['es-MX','es']
channels = {
    'h3h3Productions' : 'UCDWIvJwLJsE4LG1Atne2blQ',
}
starting_channel = 'UCDWIvJwLJsE4LG1Atne2blQ'

In [ ]:
parameters = {
    'key': api_key,
    'part': 'snippet',
    'type': 'video',
    'channelId': starting_channel,
    'maxResults': 50,
    'order': 'date'
}
max_pages = 15
query_string = urlencode(parameters)

In [ ]:
videos = {}
count = 0
for channel in channels:
    print("Searching for", channel)
    parameters['channelId'] = channels[channel]
    videos[channel] = []
    if 'pageToken' in parameters:
        del(parameters['pageToken'])
        query_string = urlencode(parameters)
    pages = max_pages
    page_token = 'FIRST TIME!'
    while pages > 0 and len(page_token) > 0:
        qurl = search_url + query_string
        print(qurl)
        r = requests.get(search_url + query_string)
        result = json.loads(r.text)
        try:
            page_token = result["nextPageToken"]
        except:
            page_token = ''
        parameters['pageToken'] = page_token
        pages = pages - 1
        print(len(result['items']), page_token)
        videos[channel].extend(result['items'])
        count += len(result['items'])
        query_string = urlencode(parameters)

In [ ]:
# Conversion to dataframes
chn = []
ids = []
pub = []
titles = []
for c in channels:
    for v in videos[c]:
        videoId = v['id']['videoId']
        publishedDate = v['snippet']['publishedAt']
        title =  v['snippet']['title']
        chn.append(slugify(c))
        ids.append(videoId)
        pub.append(publishedDate)
        titles.append(title)
initial_df = pd.DataFrame({
    'channel':chn,
    'id': ids,
    'published_at': pub,
    'title': titles
})
initial_df['published_at'] = pd.to_datetime(initial_df['published_at'])
initial_df.to_csv("youtube-captions/h3h3_initial.csv", encoding='utf-8')
print(initial_df.info())

In [ ]:
ids = list(initial_df['id'].values)
categories = []
default_language = []
durations = []
license = []
viewCounts = []
likeCounts = []
dislikeCounts = []
favoriteCounts = []
commentCounts = []

batch_size = 50
i = 0
video_details = "https://www.googleapis.com/youtube/v3/videos?id=%s&part=snippet,statistics,contentDetails&key=%s" 
while i < len(ids):
    ids_to_query = ','.join(ids[i:i+batch_size])
    q = video_details % (ids_to_query, api_key)
    r = requests.get(q)
    resultlist = json.loads(r.text)
    for result in resultlist['items']:
        snippet = result['snippet']
        contentDetails = result['contentDetails']
        statistics = result['statistics']

        categories.append(snippet['categoryId'])
        if 'defaultAudioLanguage' in snippet:
            default_language.append(snippet['defaultAudioLanguage'])
        else:
            default_language.append('-')
        durations.append(contentDetails['duration'])
        license.append(contentDetails['licensedContent'])
        viewCounts.append(statistics['viewCount'])
        favoriteCounts.append(statistics['favoriteCount'])
        likeCount = -1
        dislikeCount = -1
        commentCount = -1
        if 'likeCount' in statistics:
            likeCount = int(statistics['likeCount'])
            dislikeCount = int(statistics['dislikeCount'])
        if 'commentCount' in statistics:
            commentCount = int(statistics['commentCount'])
        likeCounts.append(likeCount)
        dislikeCounts.append(dislikeCount)
        commentCounts.append(commentCount)
    
    i += batch_size

details_df = pd.DataFrame({
    'id': ids,
    'category':categories,
    'language': default_language,
    'duration': durations,
    'license': license,
    'views': viewCounts,
    'likes': likeCounts,
    'dislikes': dislikeCounts,
    'favs': favoriteCounts,
    'comments': commentCounts
})

details_df.to_csv("youtube-captions/h3h3_details.csv", encoding='utf-8')
print(details_df.info())

In [ ]:
initial_df = pd.read_csv("youtube-captions/h3h3_initial.csv", index_col=0, parse_dates=['published_at'], na_values=[-1, ''])
details_df = pd.read_csv("youtube-captions/h3h3_details.csv", index_col=0, na_values=[-1, ''])


initial_df = initial_df.drop_duplicates()
details_df = details_df.drop_duplicates()
details_df.duration = details_df.duration.apply(lambda iso: isodate.parse_duration(iso).total_seconds())

#print(initial_df.info())
#print(details_df.info())

complete_df = pd.merge(left=initial_df, right=details_df, on='id')
complete_df.fillna(-1)
complete_df.set_index('id', inplace=True)

print(complete_df.describe())
complete_df.to_csv("youtube-captions/h3h3_complete.csv", encoding='utf-8')

In [ ]:
vids_subs = []
errors = []
for index, row in complete_df.iterrows():
    videoId = index
    title = row['title']
    subtitles = ''
    i = 0
    try:
        yt = YouTube('https://www.youtube.com/watch?v=' + videoId)
        while len(subtitles) == 0 and i < len(language_preferences):
            lang = language_preferences[i]
            if yt.captions.get_by_language_code(lang) is not None:
                subtitles = yt.captions.get_by_language_code(lang).xml_captions
            i = i + 1
    except:
        print("Error", videoId)
        errors.append(videoId)
    if len(subtitles) > 0:
        vids_subs.append({'id': videoId, 'title': title, 'captions': subtitles })
print("Done!")

In [ ]:
print(language_preferences)
## Getting subs & cleaning them
for subs in vids_subs:
    soup = BeautifulSoup(subs['captions'], "lxml")
    texts = soup.find_all('text')
    sub_entries = []
    for text in texts:
        sub_entry = {
            'duration': text.get('dur'),
            'start': text.get('start'),
            'content': BeautifulSoup(text.get_text(), "lxml").text
        }
        sub_entries.append(sub_entry)
    del(subs['captions'])
    subs['captions_parsed'] = sub_entries

In [ ]:
from os.path import join
## Now saving the good stuff
directory = "youtube-captions\\captions"
if not os.path.exists(directory):
    os.makedirs(directory)
print("Saving to", directory)
for vid in vids_subs:
    file_path = join(directory, slugify(vid['title']) + '.json')
    with open(file_path, 'w') as outfile:
        json.dump(vid, outfile, indent=4)
print("Done!")

In [ ]:
cleaning_subs = """directories = ['amlo', 'presidencia', 'epn']
for d in directories:
    directory = join('youtube-captions', d)
    for file in os.listdir(directory):
        if file.endswith("json"):
            video = None
            file1 = join(directory, file)
            with open(file1, 'r') as captions_file:
                video = json.load(captions_file)
                captions = video['captions_parsed']
                for cap in captions:
                    try:
                        cap['content'] = BeautifulSoup(cap['content'], "lxml").get_text()
                    except:
                        print("Error", file1)
            with open(join(directory, file), 'w') as captions_file:
                json.dump(video, captions_file)
print("Done!")
"""